// minuto 25 lezione 23 maggio pt2

#### Strategia d'investimento sui componenti dell'indice S&P 100 con back test - Short term reversal

La strategia short term reversal consiste nel prendere i titoli andati peggio nell'ultimo mese e comprarli, confidando nel fatto che rimbalzeranno; allo stesso tempo si vendono i titoli che nell'ultimo mese sono andati meglio.

https://www.youtube.com/watch?v=xpP048vfNrg

Strategia "long only" - compriamo i titoli andati peggio nel mese precedente e li teniamo per 1 mese.<br>
I titoli "perdenti" ("loser stocks") sono identificati come quelli il cui rendimento si colloca nel decile peggiore.

Il backtest consiste nel confrontare la performance di questa strategia negli ultimi 12 anni con quella dell'indice S&P 100.

In [48]:
import pandas as pd
import numpy as np
from datetime import date, datetime
import datetime as dt
from pandas.tseries.offsets import MonthEnd
import yfinance as yf
import matplotlib.pyplot as plt

Ci procuriamo i ticker dei 100 titoli del S&P100 su Wikipedia

https://en.wikipedia.org/wiki/S%26P_100

In [49]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')[2] # 2 perché deve leggere la TERZA tabella (0, 1, 2)

In [50]:
tickers.head()

,Symbol,Name,Sector
0,AAPL,Apple,Information Technology
1,ABBV,AbbVie,Health Care
2,ABT,Abbott Laboratories,Health Care
3,ACN,Accenture,Information Technology
4,ADBE,Adobe,Information Technology


In [51]:
tickers.replace('BRK.B', 'BRK-B', inplace=True)

In [52]:
tickers = tickers.Symbol.to_list() # prende i ticker e li inserisce in una lista.

In [53]:
tickers

['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'ADBE',
 'AIG',
 'AMD',
 'AMGN',
 'AMT',
 'AMZN',
 'AVGO',
 'AXP',
 'BA',
 'BAC',
 'BK',
 'BKNG',
 'BLK',
 'BMY',
 'BRK-B',
 'C',
 'CAT',
 'CHTR',
 'CL',
 'CMCSA',
 'COF',
 'COP',
 'COST',
 'CRM',
 'CSCO',
 'CVS',
 'CVX',
 'DE',
 'DHR',
 'DIS',
 'DOW',
 'DUK',
 'EMR',
 'F',
 'FDX',
 'GD',
 'GE',
 'GILD',
 'GM',
 'GOOG',
 'GOOGL',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'INTU',
 'JNJ',
 'JPM',
 'KHC',
 'KO',
 'LIN',
 'LLY',
 'LMT',
 'LOW',
 'MA',
 'MCD',
 'MDLZ',
 'MDT',
 'MET',
 'META',
 'MMM',
 'MO',
 'MRK',
 'MS',
 'MSFT',
 'NEE',
 'NFLX',
 'NKE',
 'NVDA',
 'ORCL',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTX',
 'SBUX',
 'SCHW',
 'SO',
 'SPG',
 'T',
 'TGT',
 'TMO',
 'TMUS',
 'TSLA',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB',
 'V',
 'VZ',
 'WFC',
 'WMT',
 'XOM']

Scarichiamo i relativi prezzi da Yahoo Finance.

In [54]:
start = '2014-5-31'
end = datetime.today() # la data di fine è oggi

In [55]:
df = yf.download(tickers, start, end)

[*********************100%%**********************]  101 of 101 completed


In [56]:
df

Price        Adj Close                                                  \
Ticker            AAPL        ABBV         ABT         ACN        ADBE   
Date                                                                     
2014-06-02   19.859392   35.876854   32.872204   68.548790   64.639999   
2014-06-03   20.140240   36.022621   32.888710   69.054863   64.089996   
2014-06-04   20.370213   36.161762   32.830902   70.387558   64.169998   
2014-06-05   20.450138   36.638790   33.161285   70.497215   65.470001   
2014-06-06   20.393904   36.506287   33.078674   70.455025   66.910004   
...                ...         ...         ...         ...         ...   
2024-05-20  191.039993  164.559998  103.209999  305.700012  484.690002   
2024-05-21  192.350006  162.929993  102.959999  303.640015  481.850006   
2024-05-22  190.899994  159.610001  104.820000  307.109985  483.929993   
2024-05-23  186.880005  158.259995  104.230003  305.929993  483.309998   
2024-05-24  189.979996  157.059998  103.949997  300.989990  475.429993   

Price                                                                  ...  \
Ticker            AIG         AMD        AMGN         AMT        AMZN  ...   
Date                                                                   ...   
2014-06-02  42.968044    3.970000   88.114914   71.144478   15.442000  ...   
2014-06-03  43.229366    3.940000   88.621872   71.410126   15.359500  ...   
2014-06-04  43.451107    4.040000   90.157898   71.643509   15.339000  ...   
2014-06-05  43.514454    4.080000   89.090981   72.223045   16.178499  ...   
2014-06-06  43.883522    4.060000   89.151535   72.778412   16.483500  ...   
...               ...         ...         ...         ...         ...  ...   
2024-05-20  78.790001  166.330002  314.540009  191.759995  183.539993  ...   
2024-05-21  78.680000  164.660004  314.850006  194.130005  183.149994  ...   
2024-05-22  78.910004  165.520004  309.410004  191.339996  183.130005  ...   
2024-05-23  77.580002  160.429993  306.500000  186.600006  181.050003  ...   
2024-05-24  78.029999  166.360001  305.839996  185.929993  180.750000  ...   

Price        Volume                                                          \
Ticker          TXN      UNH      UNP      UPS      USB         V        VZ   
Date                                                                          
2014-06-02  3379700  1986400  2625600  2914100  4951300   6364000  14364600   
2014-06-03  3955200  2841100  2845600  2319200  4222000   9011600  19683700   
2014-06-04  3497700  3552900  2354800  2822100  4594200  12589600  12696000   
2014-06-05  4038200  3453700  3489200  2514900  4625000   6597600   9855300   
2014-06-06  2965800  3286600  4104800  2103700  5222500  10002400  13917800   
...             ...      ...      ...      ...      ...       ...       ...   
2024-05-20  5282800  2476200  1130300  1812700  9935500   5460900   8665000   
2024-05-21  5489100  2521900  2967600  2750200  6315200   6578200  18327000   
2024-05-22  9119000  2299100  2634200  3590500  5802000   4202500  12374100   
2024-05-23  7125400  3055400  2815700  4437200  7452400   6115900  14105000   
2024-05-24  4354100  2527800  2304900  2993700  6155900   4339700  10418800   

Price                                     
Ticker           WFC       WMT       XOM  
Date                                      
2014-06-02  11828300  13171200   7761300  
2014-06-03   9293500  21640800   9384200  
2014-06-04  10129200  18599700   7148800  
2014-06-05  11862900  14087400  12296100  
2014-06-06  13152400  12566700   9340900  
...              ...       ...       ...  
2024-05-20  15029000  15001400  11551300  
2024-05-21  18422100  17131700  14495000  
2024-05-22  10340500  14879400  17879300  
2024-05-23  13056100  16222900  15979800  
2024-05-24   9912500  11793800  12211400  

[2514 rows x 606 columns]

In [57]:
prices = df['Adj Close']

In [58]:
prices

Ticker,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMD,AMGN,AMT,AMZN,...,TXN,UNH,UNP,UPS,USB,V,VZ,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2014-06-02,19.859392,35.876854,32.872204,68.548790,64.639999,42.968044,3.970000,88.114914,71.144478,15.442000,...,35.873734,68.182907,79.933113,75.867348,30.466051,49.775639,30.572212,38.316559,20.799505,64.774994
2014-06-03,20.140240,36.022621,32.888710,69.054863,64.089996,43.229366,3.940000,88.621872,71.410126,15.359500,...,35.873734,68.534714,79.218765,75.393532,30.509377,49.262775,30.107981,38.316559,20.785959,65.066643
2014-06-04,20.370213,36.161762,32.830902,70.387558,64.169998,43.451107,4.040000,90.157898,71.643509,15.339000,...,35.766846,69.075226,79.395317,75.058228,30.596050,49.288418,30.022461,38.279057,20.899769,64.839790
2014-06-05,20.450138,36.638790,33.161285,70.497215,65.470001,43.514454,4.080000,89.090981,72.223045,16.178499,...,36.186737,68.466057,80.402664,75.532036,30.689959,49.472576,30.101879,38.721558,20.951250,65.170372
2014-06-06,20.393904,36.506287,33.078674,70.455025,66.910004,43.883522,4.060000,89.151535,72.778412,16.483500,...,36.263096,68.577591,81.012703,75.510170,30.971653,49.654415,30.187380,38.984035,20.921444,65.850883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-20,191.039993,164.559998,103.209999,305.700012,484.690002,78.790001,166.330002,314.540009,191.759995,183.539993,...,199.199997,517.229980,243.630005,147.589996,40.889999,278.540009,40.119999,60.700001,64.180000,118.669998
2024-05-21,192.350006,162.929993,102.959999,303.640015,481.850006,78.680000,164.660004,314.850006,194.130005,183.149994,...,199.009995,523.549988,234.960007,145.419998,41.180000,275.950012,39.619999,61.459999,65.150002,117.849998
2024-05-22,190.899994,159.610001,104.820000,307.109985,483.929993,78.910004,165.520004,309.410004,191.339996,183.130005,...,202.539993,521.349976,234.710007,143.770004,40.770000,275.579987,39.790001,60.930000,65.250000,115.480003


In [59]:
mtl_ret = prices.pct_change().resample('ME').agg(lambda x: (x+1).prod() -1)
# questo è un altro modo (più chic) per calcolare i rendimenti mensili: fa il rendimento composto dei rendimenti giornalieri e si tiene l'ultimo dato del mese.
# per maggiori informazioni, chiedi a chat-gpt (che è meglio)

In [60]:
mtl_ret

Ticker,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMD,AMGN,AMT,AMZN,...,TXN,UNH,UNP,UPS,USB,V,VZ,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2014-06-30,0.034773,0.042290,0.027638,-0.005291,0.119431,0.008191,0.055416,0.016488,0.021926,0.051612,...,0.017025,0.033525,0.006137,-0.013643,0.032863,-0.013160,-0.022378,0.028773,-0.022017,0.007404
2014-07-31,0.028731,-0.065609,0.035352,-0.019297,-0.042980,-0.047637,-0.066826,0.076202,0.049011,-0.036302,...,-0.026054,-0.008562,-0.014436,-0.054257,-0.029778,0.001424,0.041548,-0.031583,-0.019848,-0.017282
2014-08-31,0.077509,0.056171,0.002849,0.022452,0.038267,0.078492,0.066496,0.099387,0.044602,0.083229,...,0.041730,0.069464,0.075892,0.009472,0.005948,0.009075,-0.011900,0.017676,0.032854,0.012259
2014-09-30,-0.017073,0.044862,-0.015388,0.003207,-0.037691,-0.034207,-0.182254,0.007749,-0.046827,-0.048962,...,-0.010170,-0.000730,0.029923,0.009863,-0.004820,0.004000,0.003412,0.008359,0.012848,-0.054394
2014-10-31,0.071960,0.106881,0.053657,0.011080,0.013441,-0.008330,-0.178886,0.154635,0.041333,-0.052661,...,0.048659,0.101565,0.074064,0.067352,0.018408,0.131509,0.016454,0.023520,-0.002615,0.028283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-31,-0.042227,0.070986,0.032948,0.040748,0.035501,0.025978,0.137575,0.091105,-0.093710,0.021456,...,-0.053247,-0.027979,-0.006881,-0.097500,-0.040203,0.049587,0.142285,0.019504,0.048208,0.028306
2024-02-29,-0.018543,0.070864,0.048520,0.029955,-0.093075,0.048626,0.148130,-0.121840,0.016407,0.138918,...,0.045029,-0.035448,0.045377,0.056537,0.010111,0.036255,-0.055018,0.115593,0.064024,0.026081
2024-03-31,-0.051286,0.034365,-0.041976,-0.075164,-0.099379,0.077549,-0.062536,0.038308,-0.006386,0.020480,...,0.041116,0.006179,-0.030589,0.002496,0.077428,-0.012596,0.048476,0.042634,0.030113,0.112132


In [61]:
mtl_ret.describe()

Ticker,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMD,AMGN,AMT,AMZN,...,TXN,UNH,UNP,UPS,USB,V,VZ,WFC,WMT,XOM
count,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,...,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000
mean,0.022143,0.015098,0.011306,0.014457,0.020005,0.008746,0.044496,0.012818,0.009966,0.024583,...,0.016425,0.018407,0.011111,0.007843,0.005161,0.015992,0.003488,0.007077,0.010986,0.007694
std,0.080159,0.074524,0.058208,0.064237,0.080433,0.083530,0.164057,0.070038,0.062336,0.089544,...,0.064472,0.056276,0.066912,0.075616,0.074094,0.058301,0.051314,0.080832,0.052975,0.078532
min,-0.181198,-0.182854,-0.151885,-0.144864,-0.263068,-0.417550,-0.410489,-0.153768,-0.154891,-0.237525,...,-0.130742,-0.114586,-0.142455,-0.172992,-0.249369,-0.116490,-0.117071,-0.297430,-0.156170,-0.261858
25%,-0.034078,-0.028433,-0.023472,-0.026264,-0.035762,-0.030242,-0.067776,-0.037350,-0.027895,-0.035891,...,-0.026291,-0.022463,-0.027821,-0.041166,-0.036380,-0.025610,-0.028235,-0.032741,-0.020632,-0.037493
50%,0.025818,0.013118,0.010668,0.024794,0.027170,0.012257,0.016854,0.012860,0.009369,0.025795,...,0.014796,0.017223,0.010431,0.009624,0.008386,0.021024,0.002189,0.013937,0.010184,0.005803
75%,0.079018,0.058280,0.046456,0.053400,0.069558,0.048899,0.169866,0.053984,0.051962,0.076734,...,0.057519,0.053763,0.058174,0.051143,0.047123,0.052423,0.039141,0.047502,0.042820,0.043777
max,0.216569,0.228907,0.171940,0.159905,0.189597,0.220705,0.471773,0.199423,0.171671,0.270596,...,0.171938,0.178456,0.231570,0.284044,0.200968,0.166113,0.142285,0.280913,0.147765,0.269156


In [62]:
formation = dt.datetime(2014,7,31)

In [63]:
formation # data di partenza

datetime.datetime(2014, 7, 31, 0, 0)

In [64]:
ret_1 = mtl_ret.loc[formation - MonthEnd(1)] # rendimento del mese precedente rispetto a formation

In [65]:
ret_1

Ticker
AAPL    0.034773
ABBV    0.042290
ABT     0.027638
ACN    -0.005291
ADBE    0.119431
          ...   
V      -0.013160
VZ     -0.022378
WFC     0.028773
WMT    -0.022017
XOM     0.007404
Name: 2014-06-30 00:00:00, Length: 101, dtype: float64

In [66]:
ret_1.name = 'mtl_ret'

In [67]:
ret_1 = pd.DataFrame(ret_1)

In [68]:
ret_1

,mtl_ret
Ticker,
AAPL,0.034773
ABBV,0.042290
ABT,0.027638
ACN,-0.005291
ADBE,0.119431
...,...
V,-0.013160
VZ,-0.022378
WFC,0.028773


In [69]:
ret_1['decile'] = pd.qcut(ret_1.mtl_ret,10,labels=False) # quantilecut è una funzione che serve ad assegnare ogni riga (rendimento) ad uno dei quantili

In [70]:
ret_1

,mtl_ret,decile
Ticker,,
AAPL,0.034773,6
ABBV,0.042290,7
ABT,0.027638,6
ACN,-0.005291,2
ADBE,0.119431,9
...,...,...
V,-0.013160,1
VZ,-0.022378,0
WFC,0.028773,6


In [71]:
losers = ret_1[ret_1.decile == 0].index # cattura i soli titoli che si trovano nel decile peggiore in un nuovo DataFrame

In [72]:
losers # DataFrame dei peggiori titoli del mese precedente

Index(['BA', 'BKNG', 'CVS', 'IBM', 'MA', 'ORCL', 'PG', 'PM', 'TMUS', 'VZ',
       'WMT'],
      dtype='object', name='Ticker')

In [73]:
losret = mtl_ret.loc[formation, mtl_ret.columns.isin(losers)].mean() # calcola la media dei rendimenti dei titoli losers

Questo rendimento è il peggiore di tutto il campione.<br>
Se andiamo un mese avanti le cose migliorano.

In [74]:
losret

0.0027202032332375374

__NB__: quanto pare è stato un mese positivo, dal momento che la media dei rendimenti del losers è positiva.

Simuliamo la strategia in tutto il periodo (120 mesi).

In [75]:
def reversal(formation):
    ret_1 = mtl_ret.loc[formation - MonthEnd(1)]
    ret_1.name = 'mtl_ret'
    ret_1 = pd.DataFrame(ret_1)
    ret_1['decile'] = pd.qcut(ret_1.mtl_ret, 10, labels=False,duplicates = 'drop')
    losers = ret_1[ret_1.decile == 0].index
    losret = mtl_ret.loc[formation, mtl_ret.columns.isin(losers)].mean()
    return losret

In [76]:
reversal(formation) # viene fuori lo stesso rendimento calcolato prima (stiamo chiamando la funzione sul primo mese)

0.0027202032332375374

In [77]:
returns = []
dates = []

for i in mtl_ret.index[2:]:
    returns.append(reversal(i))
    dates.append(i)

In [78]:
returns

[0.06708167235702574,
 -0.005856576938914332,
 -0.026012545757383695,
 0.01947422289763877,
 -0.008965894920590826,
 0.006928240676528408,
 0.09202521416415058,
 -0.011235993778643233,
 0.0730673944252611,
 0.04689736556557702,
 -0.03466166304117055,
 -0.02045393764388024,
 -0.04379246526367981,
 -0.04869397488675967,
 0.09709795237216844,
 0.0034393074354724436,
 0.018850284137368403,
 -0.09192535849791254,
 -0.0011312966729567094,
 0.0756641004073472,
 0.0459314421313451,
 0.04672701333650873,
 0.003448362594712373,
 0.06292478659414596,
 0.008724021692951308,
 -0.009186649763949218,
 -0.0015673601461302265,
 0.04122877656026083,
 0.037352940702360234,
 0.020982745181659466,
 0.0603833424688758,
 0.004359567445179321,
 0.001471263757839973,
 0.004736140365668152,
 0.03778466226922275,
 0.035108178619895264,
 0.010357860721675922,
 0.030969690340263825,
 0.015648787317153943,
 0.013596639955494245,
 -0.00656833541650138,
 0.050122234316247134,
 -0.05204090562770476,
 -0.05042447417497

In [79]:
frame =  pd.DataFrame({'dates': dates, 'returns':returns}) # creiamo un nuovo DataFrame in cui ci sono le date e i rendimenti della strategia

In [80]:
frame

,dates,returns
0,2014-08-31,0.067082
1,2014-09-30,-0.005857
2,2014-10-31,-0.026013
3,2014-11-30,0.019474
4,2014-12-31,-0.008966
...,...,...
113,2024-01-31,0.006197
114,2024-02-29,0.036851
115,2024-03-31,0.041192
116,2024-04-30,-0.049558


In [81]:
frame.returns.mean() # rendimento medio mensile della strategia

0.016420269626869897

__NB__: mentre la strategia delle medie mobili ha costi variabili ma contenuti, qui c'è la certezza quasi matematica che tutti i mesi vengono fatte molte operazioni di acquisto o vendita (con conseguenti costi).

Confrontiamo la strategia con il rendimento del benchmark.

In [82]:
start = '2014-5-31'
end = datetime.today()
df2 = yf.download('^OEX', start, end)['Adj Close'] 

[*********************100%%**********************]  1 of 1 completed


In [83]:
df2 # SP100

Date
2014-06-02     852.380005
2014-06-03     851.859985
2014-06-04     853.109985
2014-06-05     858.859985
2014-06-06     862.320007
                 ...     
2024-05-20    2525.709961
2024-05-21    2535.719971
2024-05-22    2527.840088
2024-05-23    2515.209961
2024-05-24    2533.129883
Name: Adj Close, Length: 2514, dtype: float64

In [84]:
bench_ret = df2.pct_change().resample('ME').agg(lambda x: (x+1).prod() -1) # rendimento dell'indice SP100

C:\Users\utente\AppData\Local\Temp\ipykernel_3400\3668985387.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  bench_ret = df2.pct_change().resample('M').agg(lambda x: (x+1).prod() -1)


In [85]:
bench_ret

Date
2014-06-30    0.015357
2014-07-31   -0.007661
2014-08-31    0.035210
2014-09-30   -0.008807
2014-10-31    0.016499
                ...   
2024-01-31    0.023379
2024-02-29    0.054534
2024-03-31    0.027175
2024-04-30   -0.037489
2024-05-31    0.061699
Freq: ME, Name: Adj Close, Length: 120, dtype: float64

In [86]:
frame['S&P100 ret'] = bench_ret[2:].values

In [87]:
frame

,dates,returns,S&P100 ret
0,2014-08-31,0.067082,0.035210
1,2014-09-30,-0.005857,-0.008807
2,2014-10-31,-0.026013,0.016499
3,2014-11-30,0.019474,0.022829
4,2014-12-31,-0.008966,-0.008579
...,...,...,...
113,2024-01-31,0.006197,0.023379
114,2024-02-29,0.036851,0.054534
115,2024-03-31,0.041192,0.027175
116,2024-04-30,-0.049558,-0.037489


In [88]:
frame.describe()

,dates,returns,S&P100 ret
count,118,118.000000,118.000000
mean,2019-07-16 05:53:53.898305024,0.016420,0.010201
min,2014-08-31 00:00:00,-0.232775,-0.103403
25%,2017-02-07 00:00:00,-0.008628,-0.014884
50%,2019-07-15 12:00:00,0.018861,0.015492
75%,2021-12-23 06:00:00,0.046528,0.036385
max,2024-05-31 00:00:00,0.211660,0.126773
std,NaN,0.062177,0.044684


Osservando i dati si può notare che la strategia short-term reversal ha battuto l'indice SP100 (in _assenza_ di costi di transazione), a fronte però di una volatilità più alta.

Calcoliamo i rendimenti composti sottraendo la volatilità.

In [89]:
stra_adj_ret = frame.returns.mean() - 0.5*(frame.returns.std())**2
stra_adj_ret

0.014487300830330802

In [90]:
sp100_adj_ret = frame['S&P100 ret'].mean() - 0.5*(frame['S&P100 ret'].std())**2
sp100_adj_ret

0.009202560615085046

La strategia short-term reversal continua ad essere favorita rispetto all'indice SP100.

Calcoliamo infine quante volte la strategia batte il benchmark (in percentuale).

In [91]:
ok = frame[frame.returns > frame['S&P100 ret']].shape[0]
ok

70

In [92]:
tutti = frame.shape[0]
tutti

118

In [93]:
hit_ratio = ok/tutti
hit_ratio

0.5932203389830508